# EXtra-foam azimuthal integration benchmark

In [ ]:
import os.path as osp

import numpy as np
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator as PyfaiAzimuthalIntegrator
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

import extra_foam
print(extra_foam.__version__)

from extra_foam.algorithms import AzimuthalIntegrator, ConcentricRingsFinder
from extra_foam.algorithms import mask_image_data

In [ ]:
def load_image(filepath):
    img = np.load(osp.join(osp.expanduser('~'), filepath))
    mask = np.zeros_like(img, dtype=bool)
    mask_image_data(img, threshold_mask=(0, 20000), out=mask)
    _, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(img)
    
    return img, mask

# img, mask = load_image("jf_ring.npy")
# cy, cx = -33, 1112
# pixel1, pixel2 = 75e-6, 75e-6  # pixel size (y, x)

# img, mask = load_image("jf_ring_6modules.npy")
# cy, cx = 537, 1132
# pixel1, pixel2 = 75e-6, 75e-6  # pixel size (y, x)

img, mask = load_image("lpd.npy")
cy, cx = 606, 554
pixel1, pixel2 = 200e-6, 200e-6  # pixel size (y, x)

#### Integrate a single image

In [ ]:
dist = 1  # sample distance
npt = 1024  # number of integration points
poni1, poni2 = cy * pixel1, cx * pixel2  # integration center (y, x)

In [ ]:
# %%timeit

pyfai_method = 'nosplit_csr'
pyfai_integrator = PyfaiAzimuthalIntegrator(
    dist=dist, poni1=poni1, poni2=poni2, pixel1=pixel1, pixel2=pixel2, wavelength=1e-10)

q_gt, I_gt = pyfai_integrator.integrate1d(img, npt, mask=mask, unit="q_A^-1", method=pyfai_method)

In [ ]:
%timeit q_gt, I_gt = pyfai_integrator.integrate1d(img, npt, mask=mask, unit="q_A^-1", method=pyfai_method)

In [ ]:
# %%timeit

integrator = AzimuthalIntegrator(
    dist=dist, poni1=poni1, poni2=poni2, pixel1=pixel1, pixel2=pixel2, wavelength=1e-10)

q, I = integrator.integrate1d(img, npt=npt)

In [ ]:
%timeit q, I = integrator.integrate1d(img, npt=npt)

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))

ax.plot(1e-10 * q, I, '-', label='EXtra-foam')
ax.plot(q_gt, I_gt, '--', label='pyFAI')
ax.set_xlabel("q (1/A)", fontsize=16)
ax.set_ylabel("I (arb.)", fontsize=16)
ax.legend(fontsize=16)

#### Integrate an array of images

In [ ]:
import multiprocessing as mp

print(mp.cpu_count())

In [ ]:
img_array = np.tile(img, (40, 1, 1))
print(img_array.shape)

q_a, I_a = integrator.integrate1d(img_array, npt=npt)
np.testing.assert_array_equal(q_a, q)
np.testing.assert_array_equal(I_a[0], I)
np.testing.assert_array_equal(I_a[39], I)

%timeit integrator.integrate1d(img_array, npt=npt)

#### Concentric ring finding

In [ ]:
min_count = 500
prominence = 100
distance = 10

finder = ConcentricRingsFinder(pixel2, pixel1)
cx, cy = finder.search(img, cx, cy, min_count=min_count)

In [ ]:
%timeit finder.search(img, cx, cy, min_count=min_count)

In [ ]:
q, s = finder.integrate(img, cx, cy, min_count=min_count)

i_peaks = find_peaks(s, distance=distance, prominence=prominence)[0]

_, ax = plt.subplots(figsize=(12, 6))

ax.plot(q, s, '-')
ax.plot(q[i_peaks], s[i_peaks], 'x')
ax.set_xlabel("Radial (pixel)", fontsize=16)
ax.set_ylabel("I (arb.)", fontsize=16)

print("Optimized cx = ", cx, ", cy = ", cy)